In [116]:
import gensim.downloader as api

# Using the Word2Vec semantic embedding model trained on google news
wv = api.load('word2vec-google-news-300')

INFO:gensim.models.keyedvectors:loading projection weights from /Users/ddevatha/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/ddevatha/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-08-07T13:07:50.217328', 'gensim': '4.3.3', 'python': '3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 08:03:38) [Clang 14.0.6 ]', 'platform': 'macOS-13.2.1-arm64-arm-64bit', 'event': 'load_word2vec_format'}


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

# Preprocessing the text (removing punct and stopwords)
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_tokens.append(token.lemma_.lower())
    return " ".join(filtered_tokens)

In [ ]:
from linkedin_api import Linkedin # importing linkedin profile data

api = Linkedin('deeptanshu.devatha8@gmail.com', 'Earth@123')
varun = api.get_profile('varun-koduri-653256314')
gates = api.get_profile('williamhgates')
ramesh = api.get_profile('rameshchittipolu')
schulz = api.get_profile('chadcschulz')
drexler = api.get_profile('millard-mickey-drexler-1b00a9269')
nisha = api.get_profile('nishapaliwal')
reshma = api.get_profile('reshma-kewalramani-md-fasn-8328ba21')

linkedin_db = [varun, gates, ramesh, schulz, drexler, mike, reshma]

In [ ]:
from nltk.tokenize import sent_tokenize
def print_card(candidate):
    for person in linkedin_db:
        name = ' '.join([person['firstName'], person['lastName']])
        if name == candidate:
            print()
            print(candidate)
            print(person['headline'])
            print(person['geoLocationName'], '|', 'LinkedIn Profile')
            if person['skills'] == True:
                print(person['skills'])
            try:
                sent_list = sent_tokenize(person['summary'])
                if len(sent_list) > 2:
                    print(sent_list[0], sent_list[1])
                else:
                    print(person['summary'])
            except KeyError:
                None

In [ ]:
industry_names = [] # Creating a list of dictionaries with Name: Industry pairs
for person in linkedin_db:
    name = " ".join([person['firstName'], person['lastName']])
    industry = person['industryName']
    entry = dict()
    entry[name] = preprocess(industry)
    industry_names.append(entry)

In [ ]:
from scipy.spatial.distance import cosine 

# User Interface and input vectorization
print('AI Profile Matching')
print('-' * 80)
print('Preferred Industry: ', end=" ")
industry_input = preprocess(input())
input_vector = wv.get_mean_vector(industry_input.split())

#Similarity calculation and matching logic
candidates = dict()
sim = 0
max_sim = 0
candidate = ''
for entry in industry_names:
    for k, v in entry.items():
        v_vector = wv.get_mean_vector(v.split())
        sim = 1 - cosine(input_vector, v_vector)
        max_sim = max(sim, max_sim)
        candidates[k] = sim

# Printing candidates
for k, v in candidates.items():
    if max_sim - 0.1 <= v <= max_sim + 0.1:
        print_card(k)